In [1]:
import io
import json
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
from flask import Flask,jsonify,request
import socket

In [2]:
app = Flask(__name__)

imagenet_class_index = json.load(open('image_class_index.json'))

model = torch.load('webtry.pth')
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [3]:
#image transform functions

def transform_image(image_bytes):
    my_transforms = transforms.Compose([transforms.Resize(255),transforms.CenterCrop(),transforms.ToTensor(),transforms.Normalize(
    [0.485,0.456,0.406],[0.229,0.224,0.225])])
    image = Image.open(io.BytesIO(image_bytes))
    return my_transforms(image).unsqueeze(0)

In [4]:
#get prediction

def get_prediction(image_bytes):
    tensor = transform_image(image_bytes=image_bytes)
    outputs = model.forward(tensor.to(device))
    _,y_hat = outputs.max(1)
    predicted_idx = str(y_hat,item())
    return imagenet_class_index[predicted_idx]
    

In [5]:
@app.route('/predict',methods=['POST'])
def predict():
    if request.method == 'POST':
        file = request.files['file']
        img_bytes = file.read()
        class_id , class_name = get_prediction(image_bytes=image_bytes)
        return jsonify({'class_id':class_id,'class_name':class_name})

In [6]:
import threading

threading.Thread(target=app.run,kwargs={'host':'0.0.0.0','port':80}).start()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [7]:
import requests

resp = requests.post('http://0.0.0.0/predict',files={'file':open('sample.jpg','rb')})

 * Running on http://0.0.0.0:80/ (Press CTRL+C to quit)
[2019-12-15 15:26:16,619] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "//anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "//anaconda3/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-5-4e77325a75a1>", lin

In [8]:
print(resp.text)

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>



127.0.0.1 - - [15/Dec/2019 15:27:45] "GET / HTTP/1.1" 404 -
